## Implementation

#### Library Imports and Global Variables

In [288]:
from benchmark_reader import *
import re
import pprint as pp
import random
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060


In [289]:
EN = ''
MT = 'mt'

#### Function Definitions

In [290]:
def saveData(dataset, datatype, folder):

    f_source = open(folder+"/data/src-"+datatype+".txt", "w")
    f_target = open(folder+"/data/tgt-"+datatype+".txt", "w")

    for data in dataset:

        f_source.write(' '.join(data['triple']) + '\n')
        f_target.write(data['sentence'] + '\n')

    f_source.close()
    f_target.close()

def formatData(b, language):
    
    dataset = []

    for entry in b.entries:

        lexicalisations = [] #to store lexicalisations

        if int(entry.size) == 1: #to only get sentences with one triple

            for triple in entry.modifiedtripleset.triples:

                s = triple.s
                p = triple.p
                o = triple.o

                #to handle p when it is made up of multiple words

                words = ''
                for i in range(len(p)):

                    if p[i].isupper(): #when upper case space is added then letter is added

                        words += " " + p[i]
                        
                    else: #letter is added
                        words += p[i]

                p = words

                #replacing special characters with space
                # s = re.sub(r'[,|""_:@#?!&$]', ' ', s)
                # p = re.sub(r'[,|""_:@#?!&$]', ' ', p)
                # o = re.sub(r'[,|""_:@#?!&$]', ' ', o)

                # changing to lower case
                # s = s.lower()
                # p = p.lower()
                # o = o.lower()

                triple = [s, p, o]

            for i in range(entry.count_lexs()):
                if(entry.lexs[i].lang == language): #to only get english. use 'mt' for maltese

                    lex = entry.lexs[i].lex

                    #changing to lower case
                    # lex = lex.lower()

                    lexicalisations.append(lex)

            for lex in lexicalisations:

                dataset.append({
                    'triple': triple,
                    'sentence': lex
                })

    return dataset 

def preprocess(language, folder):

    b = Benchmark() #create benchmark object
    files = select_files("xmldata/mt_train.xml") #getting file from data folder
    b.fill_benchmark(files) #parse xml files and fill Benchmark with Entry instances

    traintest_dataset = formatData(b, language) #contains data that will be used for training and testing

    print("Triple formatting: ", traintest_dataset[0]['triple'])
    print("Sentence formatting: ", traintest_dataset[0]['sentence'], "\n")

    b = Benchmark() #create benchmark object
    files = select_files("xmldata/mt_dev.xml") #getting file from data folder
    b.fill_benchmark(files) #parse xml files and fill Benchmark with Entry instances

    val_data = formatData(b, language) #contains data that will be used for validation

    random.seed(42)
    random.shuffle(traintest_dataset) #shuffling the dataset

    train_split = int(len(traintest_dataset)*0.9) #90% of data for training

    train_data = traintest_dataset[:train_split]
    test_data = traintest_dataset[train_split:]

    # create sets of triples for easy comparison
    test_triples = set([tuple(data['triple']) for data in test_data])

    # remove triples from training set that appear in the testing set
    train_data_filtered = [data for data in train_data if tuple(data['triple']) not in test_triples]

    #to increase validation data in maltese from 403
    if language == MT:

        sentencesAmount = 450 #amount of sentences to be added to validation data
        
        #take first 450 data from train_data_filtered
        new_val_data = train_data_filtered[:sentencesAmount]
        #remove first 450 data from train_data_filtered
        train_data_filtered = train_data_filtered[sentencesAmount:]

        #add new_val_data to val_data
        val_data = val_data + new_val_data

        #check if val_data has duplicates
        val_triples = set([tuple(data['triple']) for data in val_data])

        #making sure that sentences in val_data are not in train_data_filtered
        train_data_filtered = [data for data in train_data_filtered if tuple(data['triple']) not in val_triples]

    print("Training data: ", len(train_data_filtered), "\n", 
        "Validation data: ", len(val_data), "\n",
        "Testing data: ", len(test_data), sep='')

    saveData(train_data_filtered, 'train', folder)
    saveData(val_data, 'val', folder)
    saveData(test_data, 'test', folder)

### LSTM Model on English Data

In [291]:
preprocess(EN, 'data-en')

Triple formatting:  ['Aarhus_Airport', 'city Served', '"Aarhus, Denmark"']
Sentence formatting:  The Aarhus is the airport of Aarhus, Denmark. 

Training data: 5681
Validation data: 961
Testing data: 769


In [292]:
!onmt_build_vocab -config config/config-lstm-en.yaml -n_sample -1

[2023-06-01 18:57:27,935 INFO] Counter vocab from -1 samples.
[2023-06-01 18:57:27,936 INFO] n_sample=-1: Build vocab on full datasets.
[2023-06-01 18:57:29,973 INFO] Counters src: 3163
[2023-06-01 18:57:29,973 INFO] Counters tgt: 6277
[2023-06-01 18:57:29,973 WARNING] path data-en/run/example.vocab.src exists, may overwrite...
[2023-06-01 18:57:29,976 WARNING] path data-en/run/example.vocab.tgt exists, may overwrite...


In [293]:
!onmt_train -config config/config-lstm-en.yaml

[2023-06-01 18:57:32,235 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2023-06-01 18:57:32,235 INFO] Missing transforms field for valid data, set to default: [].
[2023-06-01 18:57:32,235 INFO] Parsed 2 corpora from -data.
[2023-06-01 18:57:32,235 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-01 18:57:32,269 INFO] Building model...
[2023-06-01 18:57:32,566 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(3168, 500, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (rnn): LSTM(500, 500, num_layers=2, batch_first=True, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(6288, 500, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.3, inplace=False)
  

In [294]:
!onmt_translate -model data-en/run/model_step_1000.pt -src data-en/data/src-test.txt -output data-en/results/pred_1000.txt -gpu 0 -verbose 

[2023-06-01 18:58:25,403 INFO] 
SENT 1: ['Alfred_Garth_Jones', 'death', 'Place', '<unk>']
PRED 1: Alfred Garth Jones died in New York.
PRED SCORE: -0.2942

[2023-06-01 18:58:25,404 INFO] 
SENT 2: ['Aleksandre_Guruli', 'club', '<unk>']
PRED 2: Aleksandre Guruli played for the Olympique Lyonnais club.
PRED SCORE: -0.5493

[2023-06-01 18:58:25,404 INFO] 
SENT 3: ['Bakewell_pudding', 'creator', '<unk>', '<unk>', '<unk>', 'in', '<unk>']
PRED 3: Bakewell pudding was the former place of Bakewell pudding
PRED SCORE: -0.3876

[2023-06-01 18:58:25,404 INFO] 
SENT 4: ['Ardmore_Airport_(New_Zealand)', '3rd', 'Runway', 'Length', 'Feet', '<unk>']
PRED 4: The runway name of Ardmore Airport (New Zealand) is 07/25.
PRED SCORE: -0.5458

[2023-06-01 18:58:25,404 INFO] 
SENT 5: ['Texas', 'largest', 'City', 'Houston']
PRED 5: Abilene is the largest city in Texas.
PRED SCORE: -0.5433

[2023-06-01 18:58:25,405 INFO] 
SENT 6: ['Manila', 'leader', 'Title', '<unk>', '<unk>']
PRED 6: The leader name of Atlantic 

### LSTM Model on English Data with GloVe Embeddings

In [295]:
preprocess(EN, 'data-en')

Triple formatting:  ['Aarhus_Airport', 'city Served', '"Aarhus, Denmark"']
Sentence formatting:  The Aarhus is the airport of Aarhus, Denmark. 

Training data: 5681
Validation data: 961
Testing data: 769


In [296]:
!onmt_build_vocab -config config/config-lstm-en-glove.yaml -n_sample -1

[2023-06-01 18:58:31,835 INFO] Counter vocab from -1 samples.
[2023-06-01 18:58:31,835 INFO] n_sample=-1: Build vocab on full datasets.
[2023-06-01 18:58:34,037 INFO] Counters src: 3163
[2023-06-01 18:58:34,037 INFO] Counters tgt: 6277
[2023-06-01 18:58:34,037 WARNING] path data-en/run-glove/example.vocab.src exists, may overwrite...
[2023-06-01 18:58:34,040 WARNING] path data-en/run-glove/example.vocab.tgt exists, may overwrite...


In [297]:
!onmt_train -config config/config-lstm-en-glove.yaml

[2023-06-01 18:58:36,380 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2023-06-01 18:58:36,381 INFO] Missing transforms field for valid data, set to default: [].
[2023-06-01 18:58:36,381 INFO] Parsed 2 corpora from -data.
[2023-06-01 18:58:36,381 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-01 18:58:36,415 INFO] Reading decoder embeddings from glove-embeddings/glove.6B.300d.txt
[2023-06-01 18:58:41,547 INFO] 	Found 400000 total vectors in file
[2023-06-01 18:58:41,547 INFO] After filtering to vectors in vocab:
[2023-06-01 18:58:41,548 INFO] 	* dec: 1369 match, 4919 missing, (21.77%)
[2023-06-01 18:58:41,548 INFO] 
Saving decoder embeddings as:
	* dec: data-en/run-glove/example.dec_embeddings.pt
[2023-06-01 18:58:42,584 INFO] Building model...
[2023-06-01 18:58:42,882 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(31

In [298]:
!onmt_translate -model data-en/run-glove/model_step_1000.pt -src data-en/data/src-test.txt -output data-en/results/pred_1000_glove.txt -gpu 0 -verbose

[2023-06-01 18:59:35,081 INFO] 
SENT 1: ['Alfred_Garth_Jones', 'death', 'Place', '<unk>']
PRED 1: Alfred Garth Jones died in New York.
PRED SCORE: -0.1903

[2023-06-01 18:59:35,081 INFO] 
SENT 2: ['Aleksandre_Guruli', 'club', '<unk>']
PRED 2: Aleksandre Guruli played for the Italian national under-19 football team.
PRED SCORE: -0.2975

[2023-06-01 18:59:35,081 INFO] 
SENT 3: ['Bakewell_pudding', 'creator', '<unk>', '<unk>', '<unk>', 'in', '<unk>']
PRED 3: Bananaman is the creator of Bananaman.
PRED SCORE: -0.7506

[2023-06-01 18:59:35,081 INFO] 
SENT 4: ['Ardmore_Airport_(New_Zealand)', '3rd', 'Runway', 'Length', 'Feet', '<unk>']
PRED 4: The Ariane 5 has a runway length of 3990.0.
PRED SCORE: -0.5062

[2023-06-01 18:59:35,081 INFO] 
SENT 5: ['Texas', 'largest', 'City', 'Houston']
PRED 5: Arabic is the largest city in Texas.
PRED SCORE: -0.3358

[2023-06-01 18:59:35,082 INFO] 
SENT 6: ['Manila', 'leader', 'Title', '<unk>', '<unk>']
PRED 6: The Mayor of Denmark is the leader of Brazil.
P

### LSTM Model on Maltese Data

In [299]:
preprocess(MT, 'data-mt')

Triple formatting:  ['Aarhus_Airport', 'city Served', '"Aarhus, Denmark"']
Sentence formatting:  L-Aarhus huwa l-ajruport ta’ Aarhus, id-Danimarka. 

Training data: 4536
Validation data: 853
Testing data: 769


In [300]:
!onmt_build_vocab -config config/config-lstm-mt.yaml -n_sample -1

[2023-06-01 18:59:40,644 INFO] Counter vocab from -1 samples.
[2023-06-01 18:59:40,644 INFO] n_sample=-1: Build vocab on full datasets.
[2023-06-01 18:59:42,812 INFO] Counters src: 2783
[2023-06-01 18:59:42,812 INFO] Counters tgt: 7395
[2023-06-01 18:59:42,812 WARNING] path data-mt/run/example.vocab.src exists, may overwrite...
[2023-06-01 18:59:42,814 WARNING] path data-mt/run/example.vocab.tgt exists, may overwrite...


In [301]:
!onmt_train -config config/config-lstm-mt.yaml

[2023-06-01 18:59:45,457 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2023-06-01 18:59:45,458 INFO] Missing transforms field for valid data, set to default: [].
[2023-06-01 18:59:45,458 INFO] Parsed 2 corpora from -data.
[2023-06-01 18:59:45,458 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2023-06-01 18:59:45,500 INFO] Building model...
[2023-06-01 18:59:45,821 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(2792, 500, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (rnn): LSTM(500, 500, num_layers=2, batch_first=True, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(7400, 500, padding_idx=1)
        )
      )
      (dropout): Dropout(p=0.3, inplace=False)
  

In [302]:
!onmt_translate -model data-mt/run/model_step_1000.pt -src data-mt/data/src-test.txt -output data-mt/results/pred_1000.txt -gpu 0 -verbose

[2023-06-01 19:00:35,514 INFO] 
SENT 1: ['Alfred_Garth_Jones', 'death', 'Place', '<unk>']
PRED 1: James Garth Jones meret fl-Ingilterra.
PRED SCORE: -0.7469

[2023-06-01 19:00:35,515 INFO] 
SENT 2: ['Aleksandre_Guruli', 'club', '<unk>']
PRED 2: Aleksandre Guruli ħaddem għall-klub Lesquin.
PRED SCORE: -0.5439

[2023-06-01 19:00:35,515 INFO] 
SENT 3: ['Bakewell_pudding', 'creator', '<unk>', '<unk>', '<unk>', 'in', '<unk>']
PRED 3: L-ewwel tal-Bakewell huwa magħmul isem
PRED SCORE: -1.2668

[2023-06-01 19:00:35,515 INFO] 
SENT 4: ['Ardmore_Airport_(New_Zealand)', '3rd', 'Runway', 'Length', 'Feet', '<unk>']
PRED 4: It-tul runway runway fl-ajruport ta’ Ardmore, New Zealand huwa msejħa Zealand.
PRED SCORE: -0.7613

[2023-06-01 19:00:35,515 INFO] 
SENT 5: ['Texas', 'largest', 'City', 'Houston']
PRED 5: Austin hija l-akbarbelt tat-Tessas.
PRED SCORE: -0.5622

[2023-06-01 19:00:35,515 INFO] 
SENT 6: ['<unk>', 'leader', 'Title', '<unk>', '<unk>']
PRED 6: Il-parti huwa mmexxija tal-mexxej tal-Bel

## Evaluation

### Different ways of preprocessing data

#### English

`python 2023-Challenge-main\evaluation\automatic\scripts\eval.py -hyp data-en/results/pred_1000.txt -ref data-en/data/tgt-test.txt -nr 1 -m bleu -lng en`

<b>1. Data is kept as is:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus_Airport', 'cityServed', '"Aarhus, Denmark"'] <br>
Sentence formatting: &nbsp;The Aarhus is the airport of Aarhus, Denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 16.4104  |



<b>2. Regular expression is used to remove special characters:</b><br><br>
Triple formatting:  &emsp;&emsp;['Aarhus Airport', 'cityServed', ' Aarhus  Denmark '] <br>
Sentence formatting:  &nbsp;The Aarhus is the airport of Aarhus, Denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 16.3489  |


<b>3. Everything is set to lowercase:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus_airport', 'cityserved', '"aarhus, denmark"'] <br>
Sentence formatting: &nbsp;the aarhus is the airport of aarhus, denmark.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 14.6464  |


<b>4. Splitting the predicate into separate words:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus_Airport', 'city Served', '"Aarhus, Denmark"'] <br>
Sentence formatting: &nbsp;The Aarhus is the airport of Aarhus, Denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 20.1420  |


<b>5. RE + Lowercase:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus airport', 'cityserved', ' aarhus  denmark '] <br>
Sentence formatting: &nbsp;the aarhus is the airport of aarhus, denmark.   <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 14.9612  |


<b>6. RE + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus Airport', 'city Served', ' Aarhus  Denmark ']  <br>
Sentence formatting: &nbsp;The Aarhus is the airport of Aarhus, Denmark.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 15.3697  |


<b>7. Lowercase + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus_airport', 'city served', '"aarhus, denmark"'] <br>
Sentence formatting: &nbsp;the aarhus is the airport of aarhus, denmark.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 15.3500  |


<b>8. RE + Lowercase + Splitting:</b><br><br>
Triple formatting:  &emsp;&emsp;['aarhus airport', 'city served', ' aarhus  denmark '] <br>
Sentence formatting: &nbsp;the aarhus is the airport of aarhus, denmark.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 15.6839  |


#### English with GloVE


`python 2023-Challenge-main\evaluation\automatic\scripts\eval.py -hyp data-en/results/pred_1000_glove.txt -ref data-en/data/tgt-test.txt -nr 1 -m bleu -lng en`

<b>1. Data is kept as is:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus_Airport', 'cityServed', '"Aarhus, Denmark"'] <br>
Sentence formatting: &nbsp;The Aarhus is the airport of Aarhus, Denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 18.0305  |



<b>2. Regular expression is used to remove special characters:</b><br><br>
Triple formatting:  &emsp;&emsp;['Aarhus Airport', 'cityServed', ' Aarhus  Denmark '] <br>
Sentence formatting:  &nbsp;The Aarhus is the airport of Aarhus, Denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 13.7654  |


<b>3. Everything is set to lowercase:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus_airport', 'cityserved', '"aarhus, denmark"'] <br>
Sentence formatting: &nbsp;the aarhus is the airport of aarhus, denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 16.2314  |


<b>4. Splitting the predicate into separate words:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus_Airport', 'city Served', '"Aarhus, Denmark"'] <br>
Sentence formatting: &nbsp;The Aarhus is the airport of Aarhus, Denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 17.4939  |


<b>5. RE + Lowercase:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus airport', 'cityserved', ' aarhus  denmark '] <br>
Sentence formatting: &nbsp;the aarhus is the airport of aarhus, denmark.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 12.2937  |


<b>6. RE + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus Airport', 'city Served', ' Aarhus  Denmark '] <br>
Sentence formatting: &nbsp;The Aarhus is the airport of Aarhus, Denmark.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 13.3875  |


<b>7. Lowercase + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp; <br>
Sentence formatting: &nbsp; <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 17.7529  |


<b>8. RE + Lowercase + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus airport', 'city served', ' aarhus  denmark '] <br>
Sentence formatting: &nbsp; the aarhus is the airport of aarhus, denmark. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 14.3805  |


#### Maltese

`python 2023-Challenge-main\evaluation\automatic\scripts\eval.py -hyp data-mt/results/pred_1000.txt -ref data-mt/data/tgt-test.txt -nr 1 -m bleu -lng mt`

<b>1. Data is kept as is:</b><br><br>
Triple formatting:  &emsp;&emsp; ['Aarhus_Airport', 'cityServed', '"Aarhus, Denmark"']<br>
Sentence formatting: &nbsp;L-Aarhus huwa l-ajruport ta’ Aarhus, id-Danimarka. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 7.34072  |



<b>2. Regular expression is used to remove special characters:</b><br><br>
Triple formatting:  &emsp;&emsp;['Aarhus Airport', 'cityServed', ' Aarhus  Denmark '] <br>
Sentence formatting:  &nbsp;L-Aarhus huwa l-ajruport ta’ Aarhus, id-Danimarka. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 9.46474  |

<b>3. Everything is set to lowercase:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus_airport', 'cityserved', '"aarhus, denmark"'] <br>
Sentence formatting: &nbsp;l-aarhus huwa l-ajruport ta’ aarhus, id-danimarka.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     |  7.6331  |


<b>4. Splitting the predicate into separate words:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus_Airport', 'city Served', '"Aarhus, Denmark"'] <br>
Sentence formatting: &nbsp;L-Aarhus huwa l-ajruport ta’ Aarhus, id-Danimarka.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 7.34072  |


<b>5. RE + Lowercase:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus airport', 'cityserved', ' aarhus  denmark '] <br>
Sentence formatting: &nbsp; l-aarhus huwa l-ajruport ta’ aarhus, id-danimarka. <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 4.71897  |


<b>6. RE + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp;['Aarhus Airport', 'city Served', ' Aarhus  Denmark '] <br>
Sentence formatting: &nbsp;L-Aarhus huwa l-ajruport ta’ Aarhus, id-Danimarka.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 7.90859  |


<b>7. Lowercase + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp; ['aarhus_airport', 'city served', '"aarhus, denmark"']<br>
Sentence formatting: &nbsp;l-aarhus huwa l-ajruport ta’ aarhus, id-danimarka.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 8.98513  |

<b>8. RE + Lowercase + Splitting:</b><br><br>
Triple formatting: &emsp;&emsp;['aarhus airport', 'city served', ' aarhus  denmark '] <br>
Sentence formatting: &nbsp;l-aarhus huwa l-ajruport ta’ aarhus, id-danimarka.  <br>

| Metric   | Score    |
|----------|----------|
| BLEU     | 9.68978  |
     


#### Tabulated Results for BLEU scores

|                  | 1         | 2      | 3     | 4         | 5     | 6      | 7     | 8         |
|------------------|-----------|--------|-------|-----------|-------|--------|-------|-----------|
| English          |16.4104    |13.3489 |14.6464|**20.1420**|14.9612|15.3697 |15.3500|15.6839    |
| English w/GloVe  |**18.0305**|13.7654 |16.2314|17.4939    |12.2937|13.3875 |17.752 |14.3805    |
| Maltese          |7.34072    |9.46474 |7.6331 |7.34072    |4.71897|7.90859 |8.98513|**9.68978**|